In [10]:
oc_login = "oc login --token=sha256~t9ivWmP9l7_t5hHg7T2NjmRcKOb216yPKRFzDBSR3Sk --server=https://c100-e.au-syd.containers.cloud.ibm.com:32526"
project_name = 'databand'
tar_file_name = "databand-v1.0.15.5-images.tar"

In [3]:
import time
start_time = time.time()

import os
from subprocess import PIPE, run

def out(command):
    result = run(command, stdout=PIPE, stderr=PIPE, universal_newlines=True, shell=True)
    return result.stdout

In [2]:
oc_login_output = out(oc_login)
print(oc_login_output)
print("Patching image repository...")

#patch_ = out('''oc patch configs.imageregistry.operator.openshift.io/cluster --patch '{"spec":{"defaultRoute":true}}' --type=merge''')

print("Getting image_route...")
image_registry_route = out("oc get route -n openshift-image-registry")
image_registry_route = image_registry_route[image_registry_route.index("default-route-openshift"):image_registry_route.index("cloud")+5]
print(image_registry_route)


print("creating a new project for databand...")
create_project = out('''oc new-project ''' + project_name + ''' --display-name ''' + project_name)

print('loading images')
#docker_load = out('''docker load -i ../''' + tar_file_name)
#print(docker_load)

print("preparing for image push...")
who_am_i = out("oc whoami")
who_am_i = who_am_i.replace('\n', '')

token = out('oc whoami -t')
token = token.replace('\n', '')

Logged into "https://c104-e.us-south.containers.cloud.ibm.com:30375" as "IAM#catherine.cao@ibm.com" using the token provided.

You have access to 64 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "databand".

Patching image repository...
Getting image_route...
default-route-openshift-image-registry.itzroks-50avscmg0p-rtowsd-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-south.containers.appdomain.cloud
creating a new project for databand...
loading images
preparing for image push...


In [6]:
print("docke login....")
docker_login = out("docker login " + image_registry_route + " -u " + who_am_i + " -p " + token)
print(docker_login)

version_tag = tar_file_name[9:18]
image_repo = image_registry_route + "/" + project_name + "/"
image_names =["dbnd-ml-trainer:python-"+version_tag, 
              "dbnd-datasource-monitor:python-"+version_tag,
              "dbnd-datastage-monitor:python-"+version_tag,
              "dbnd-dbt-monitor:python-"+version_tag,
              "dbnd-webserver:python-"+version_tag,
              "dbnd-alert-defs-syncer:python-"+version_tag,
              "dbnd-webapp:nginx-"+version_tag 
 ]

print("the following " + str(len(image_names)) + " will be pushed to the target repository")
print(image_names)
num_image = str(len(image_names))

for i in range(len(image_names)):
    print("pushing " + str(i+1) + "/" + num_image)
    cmd_tag = "docker tag " + "registry.gitlab.com/databand/databand-deploy/" + image_names[i] + " " + image_repo + image_names[i]
    print(cmd_tag)
    output = os.popen(cmd_tag)
    print(output.read())
    print("******************")
    cmd_push = "docker push " + image_repo + image_names[i]
    print(cmd_push)
    output = os.popen(cmd_push)
    print(output.read())

docke login....
Login Succeeded

the following 7 will be pushed to the target repository
['dbnd-ml-trainer:python-v1.0.15.5', 'dbnd-datasource-monitor:python-v1.0.15.5', 'dbnd-datastage-monitor:python-v1.0.15.5', 'dbnd-dbt-monitor:python-v1.0.15.5', 'dbnd-webserver:python-v1.0.15.5', 'dbnd-alert-defs-syncer:python-v1.0.15.5', 'dbnd-webapp:nginx-v1.0.15.5']
pushing 1/7
docker tag registry.gitlab.com/databand/databand-deploy/dbnd-ml-trainer:python-v1.0.15.5 default-route-openshift-image-registry.itzroks-50avscmg0p-rtowsd-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-south.containers.appdomain.cloud/databand-v2/dbnd-ml-trainer:python-v1.0.15.5

******************
docker push default-route-openshift-image-registry.itzroks-50avscmg0p-rtowsd-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-south.containers.appdomain.cloud/databand-v2/dbnd-ml-trainer:python-v1.0.15.5
The push refers to repository [default-route-openshift-image-registry.itzroks-50avscmg0p-rtowsd-6ccd7f378ae819553d37d5f2ee142bd6-0000.us-sou

In [18]:
print("generatig keys...")
fernet = out('dd if=/dev/urandom bs=32 count=1 2>/dev/null | openssl base64')
fernet = fernet.replace("\n", "")

web_server_key = out('head -c 32 /dev/urandom | base64 | tr -d =')
web_server_key = web_server_key.replace("\n", "")

#os.system('cp "user-values.yaml.example" "user-values.yaml"') 

print("configuring user-values.yaml...")
import yaml

with open("user-values.yaml") as f:
     list_doc = yaml.safe_load(f)

list_doc['global']['databand']['image']['repository'] = image_repo[:-1]
list_doc['global']['databand']['imageCredentials']['registry'] = image_registry_route
list_doc['global']['databand']['imageCredentials']['username'] = who_am_i
list_doc['global']['databand']['imageCredentials']['password'] = token
list_doc['databand']['fernetKey'] = fernet
list_doc['web']['secret_key'] = web_server_key
with open("user-values.yaml", "w") as f:
    yaml.dump(list_doc, f)

print("preparing for helm install...")
helm_install = out('helm upgrade databand --install --create-namespace --namespace ' + project_name + ' --values ./values-ocp.yaml --values ./user-values.yaml .')
print(helm_install)
expose_svc = out('oc expose svc databand-web')
print(expose_svc)

print("Databand " + version_tag + " has been installed to project " + project_name)

print("--- %s mins ---" % (round((time.time() - start_time) / 60, 2)) )

generatig keys...
configuring user-values.yaml...
preparing for helm install...
Release "databand" has been upgraded. Happy Helming!
NAME: databand
LAST DEPLOYED: Tue Jan 31 19:30:14 2023
NAMESPACE: databand-v2
STATUS: deployed
REVISION: 2
TEST SUITE: None
NOTES:
Congratulations. You have just deployed Databand

1. Access the Databand Web UI by running these commands:

   export SERVICE_NAME=$(kubectl get svc --namespace databand-v2 -l "component=web,app=databand" -o jsonpath="{.items[0].metadata.name}")
   kubectl port-forward --namespace databand-v2 svc/$SERVICE_NAME 8090:8090

   Databand Web UI is available in your web browser via http://127.0.0.1:8090


Databand v1.0.15.5 has been installed to project databand-v2
--- 578.0992252826691 seconds ---
